<a href="https://colab.research.google.com/github/xyhan-github/STATS335_F23/blob/master/hw8/TabularDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
# install and import Huggingface Datasets
! pip install datasets
from datasets import load_dataset

# load necessary functions
# Numpy
import numpy as np

# Scikit-Learn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# Pandas
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.0 MB/s eta 0:00:00


# Experiment Settings and Parameters

In [29]:
# Experiment specification
dataset_name = 'covertype' # 'adult_income', 'housing', or 'covertype'
method_name  = 'Linear' # 'Linear', 'XGBoost', or 'LightGBM'

# Method configs
config = {"depth": 2,
          "lambda": 0.25,
          "learning_rate": 0.25,
          "num_rounds": 50}

# Dataloading

In [30]:
# Load Dataset

# Load the necessary dataframes
if dataset_name == 'adult_income':
  dataset = load_dataset("scikit-learn/adult-census-income")
  dataset = dataset['train']
  df = pd.DataFrame(dataset)
  y_df = df[['income']]
  X_df = df.drop('income', axis=1)

elif dataset_name == 'housing':
  dataset = load_dataset("leostelon/california-housing")
  dataset = dataset['train']
  df = pd.DataFrame(dataset)
  y_df = df[['median_house_value']]
  X_df = df.drop('median_house_value', axis=1)

elif dataset_name == 'covertype':
  dataset = load_dataset("mstz/covertype", "covertype")
  dataset = dataset['train']

  # Trick to load big datasets faster by loading in chunks
  from tqdm import tqdm
  batch_size = 10000
  dfs = []
  for i in tqdm(range(0, len(dataset), batch_size)):
      batch = dataset[i:i + batch_size]
      dfs.append(pd.DataFrame(batch))

  df = pd.concat(dfs, ignore_index=True)
  y_df = df[['cover_type']]
  X_df = df.drop('cover_type', axis=1)
else:
  raise Exception("Invalid Dataset Name!")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

100%|██████████| 59/59 [00:20<00:00,  2.88it/s]


# Data Cleaning

In [31]:
#@title Define `prepare_data(X_df, y_df)` function
def prepare_data(X_df, y_df):

  """
  Preprocesses and prepares the input data and target labels for machine learning tasks.

  Parameters:
      X_df (pd.DataFrame): DataFrame containing the input features, including categorical columns.
      y_df (pd.DataFrame): DataFrame containing the target labels.

  Returns:
      X (np.ndarray): NumPy array of preprocessed input features after one-hot encoding categorical variables.
      y (np.ndarray): NumPy array of target labels, either as a numeric array (for regression) or encoded as
                      integers (for classification).
      obj_type (str): Indicates the type of the machine learning problem:
                      - 'reg' for regression (numeric target)
                      - 'bin' for binary classification
                      - 'mult' for multiclass classification
      num_classes (int or None): Number of classes for classification tasks. None for regression.

  The function first one-hot encodes categorical columns in X_df, then converts y_df into an appropriate format based on
  its data type:
  - If y_df contains numeric values, it's treated as a regression problem.
  - If y_df contains categorical values (including strings), it's treated as a classification problem. The target
    labels are encoded as integers, and the number of classes is determined. If there are 2 classes, it's binary
    classification; otherwise, it's multiclass classification.

  Example:
      X, y, obj_type, num_classes = prepare_data(X_df, y_df)
  """

  # Remove rows with NaNs
  X_df = X_df.dropna()
  y_df = y_df.loc[X_df.index]

  # Convert categorical columns into 0-1 variables
  object_cols = X_df.select_dtypes(include='object').columns
  X_df_encoded = pd.get_dummies(X_df, columns=object_cols)

  # Create data array
  X = X_df_encoded.values

  # Convert y into target array
  y_array = y_df.iloc[:, 0].to_numpy()

  # Create target vector
  if np.issubdtype(y_array.dtype, np.number):
      y = y_array
      obj_type = 'reg'
      num_classes = None
  else:
      # If y is categorical (including strings), use LabelEncoder for encoding
      encoder = LabelEncoder()
      y = encoder.fit_transform(y_array)
      num_classes = len(encoder.classes_)
      obj_type = 'bin' if num_classes == 2 else 'mult'

  return X, y, obj_type, num_classes

In [32]:
# Prepare the data
X, y, obj_type, num_classes = prepare_data(X_df, y_df)

In [33]:
# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Run Regressor or Classifier

In [34]:
# Run method

if method_name == "Linear":
  from sklearn.linear_model import LinearRegression, LogisticRegression

  params = {'C': 1/config["lambda"],  # Inverse of regularization strength
            'max_iter': config["num_rounds"]}  # Number of training iterations

  if obj_type == 'reg':
      model = LinearRegression()
  elif obj_type == 'bin':
      model = LogisticRegression(**params)
  elif obj_type == 'mult':
      params['multi_class'] = 'multinomial'
      model = LogisticRegression(**params)

elif method_name == "XGBoost":
  import xgboost as xgb
  params = {'learning_rate':config["learning_rate"],
            'reg_lambda':config["lambda"],
            'max_depth':config["depth"],
              'n_estimators':config["num_rounds"]}

  # set objective
  if obj_type == 'reg':
    params['objective'] = 'reg:squarederror'
  elif obj_type == 'bin':
    params['objective'] = 'binary:logistic'
  elif obj_type == 'mult':
    params['objective'] = 'multi:softprob'
    params['num_classes'] = num_classes

  model = xgb.XGBRegressor(**params) if obj_type == 'reg' else xgb.XGBClassifier(**params)

elif method_name == "LightGBM":
  import lightgbm as lgb

  params = {'learning_rate':config["learning_rate"],
            'lambda_l2':config["lambda"],
            'max_depth':config["depth"],
            'n_estimators':config["num_rounds"]}

  # set objective
  if obj_type == 'reg':
    params['objective'] = 'regression'
  elif obj_type == 'bin':
    params['objective'] = 'binary'
  elif obj_type == 'mult':
    params['objective'] = 'multiclass'
    params['num_classes'] = num_classes

  model = lgb.LGBMRegressor(**params) if obj_type == 'reg' else lgb.LGBMClassifier(**params)
else:
  raise Exception("Invalid Method Name!")
model.fit(X_train, y_train)

# Make predictions on the test set
test_preds = model.predict(X_test)
if obj_type in ['bin', 'mult']:
  test_predictions = [1 if x > 0.5 else 0 for x in test_preds]
  result = accuracy_score(y_test, test_predictions)
elif obj_type == 'reg':
  result = mean_squared_error(y_test, test_preds)

print(result)

1.3261191202506293
